# fit.kn2all.ipynb
- fitting kilonova observational data to the other models

In [17]:
import glob, os
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack
import matplotlib as mpl
from astropy import units as u
import speclite.filters
import time
import sncosmo

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 200
plt.rc('font', family='serif')

In [19]:
from helper import makeSpecColors
from helper import convert_flam2fnu
from helper import get_wollaeger
from helper import convert_app2abs
from helper import convert_abs2app
kncbtbl = get_wollaeger()
from helper import get_bandwidth_table
bdwtbl = get_bandwidth_table()
#	speclite
from helper import get_speclite_med
meds = get_speclite_med()
mlam = meds.effective_wavelengths
mbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Med']*u.Angstrom
from helper import get_speclite_sdss
sdss = get_speclite_sdss()
slam = sdss.effective_wavelengths
sbdw = bdwtbl['bandwidth'][bdwtbl['group']=='SDSS']*u.Angstrom
from helper import get_speclite_jc
jc = get_speclite_jc()
jclam = jc.effective_wavelengths
jcbdw = bdwtbl['bandwidth'][bdwtbl['group']=='Johnson Cousin']*u.Angstrom
from helper import get_speclite_lsst
lsst = get_speclite_lsst()
lsstlam = lsst.effective_wavelengths
from helper import get_lsst_depth
from helper import get_kmtnet_depth
from helper import get_7dt_depth
from helper import get_7dt_broadband_depth

from helper import get_speclite_lsst
lsst = get_speclite_lsst()
lsstlam = lsst.effective_wavelengths
from helper import get_lsst_depth
from helper import get_kmtnet_depth
from helper import get_lsst_bandwidth
lsstbdw = get_lsst_bandwidth()

# Function

In [20]:
def make_sncosmo_model2synphot(model, speclite_class, filterlist, bandlam, bandwidth, times, wave,):

	#	Photometry Table
	phtbl = Table()
	phtbl['t'] = times*u.day
	for filte in filterlist:
		phtbl[f"{filte}"] = 0.0
		phtbl[f"{filte}"].format = '1.3f'
		phtbl[f"{filte}"].unit = u.ABmag

	for tt, t in enumerate(times):
		#	Model
		flux = model.flux(t, wave)
		pflux, pwave = speclite_class.pad_spectrum(flux, wave)
		mags = speclite_class.get_ab_magnitudes(pflux, pwave)

		for key in speclite_class.names:
			mag = mags[key].item()
			filte = key.split('-')[-1]
			phtbl[f"{filte}"][tt] = mag

	return phtbl

In [21]:
def plot_synphot():
	tmin = model.mintime()
	tmax = model.maxtime()
	lmin = model.minwave()
	lmax = model.maxwave()

	flux = model.flux(0, wave)
	fnu = convert_flam2fnu(flux*(u.erg/u.second/u.Angstrom/u.cm**2), wave*u.Angstrom)
	abmag = fnu.to(u.ABmag)

	#	Med-bands
	mmags = meds.get_ab_magnitudes(flux, wave)
	medmag = mmags.as_array().item()*u.ABmag

	mean_mag = np.mean(list(mmags.as_array().item()))
	yl, yu = mean_mag-2, mean_mag+2

	#	LSST
	pflux, pwave = lsst.pad_spectrum(flux, wave)
	lsstmags = lsst.get_ab_magnitudes(pflux, pwave)
	lsstmag = lsstmags.as_array().item()*u.ABmag

	plt.plot(wave, abmag, c='grey', lw=3, alpha=0.5, label='model')
	plt.errorbar(mlam, medmag, xerr=mbdw/2, marker='s', ls='none', label=f'med ({len(medmag)})')
	plt.errorbar(lsstlam, lsstmag, xerr=lsstbdw/2, marker='s', ls='none', label='ugri')

	plt.xlim([xl, xr])
	plt.ylim([yu, yl])
	plt.legend()
	plt.xlabel('Wavelength'+r' [$\rm \AA$]')
	plt.ylabel('App. mag [AB]')

In [22]:
def plot_sncosmo_sed(model, wave, times, xl=3000, xr=10000):
	colors = makeSpecColors(len(times), palette='viridis')
	for tt, t in enumerate(times):
		#	Model
		flux = model.flux(t, wave)
		#	Plot
		plt.plot(wave, flux, c=colors[tt], lw=3, alpha=0.75, label=f"t={t} d")

	plt.xlim([xl, xr])
	plt.legend(ncol=2)
	plt.xlabel('Wavelength'+r' [$\rm \AA$]')
	plt.ylabel(r'Flux [$\rm erg/s/cm^{2}/\AA$]')

- Setting

In [23]:
path_model = '../3.table/sn_sim_sncosmo_synphot'
if ~os.path.exists(path_model): os.system(f"mkdir {path_model}")

mkdir: ../3.table/sn_sim_sncosmo_synphot: File exists


In [24]:
sctbl = ascii.read('../3.table/sncosmo.tsv')
sctbl

type,event,model,version
str3,str6,str15,float64
IIP,1999em,v19-1999em-corr,1.0
IIP,2004et,v19-2004et-corr,1.0
IIL,Model,nugent-sn2l,1.2
IIn,2007pk,v19-2007pk-corr,1.0
IIn,2009ip,v19-2009ip-corr,1.0
Ic,2004gt,v19-2004gt-corr,1.0
Ic,2007gr,v19-2007gr-corr,1.0
Ib,Model,nugent-sn1bc,1.1
IIb,1993J,v19-1993j-corr,1.0


In [25]:
xl, xr = 3000, 10000 # [A]
zlist = [1e-4, 1e-3, 1e-2, 1e-1, 1e-0]
zlist

[0.0001, 0.001, 0.01, 0.1, 1.0]

# Specific Functions

# Main

In [26]:
tag = 'lsst'
filterlist = ['u', 'g', 'r', 'i', 'z', 'y']
peakabsfilter = 'bessellb'
peakabsmag = -19.0
# z = 1e-4
zlist = [1e-4, 1e-3, 1e-2, 1e-1, 1e-0]
zz = 0
z = zlist[zz]

In [27]:
ii = 0
mdlname = sctbl['model'][ii].item()
ver = str(sctbl['version'][ii].item())
typ = sctbl['type'][ii].item()
evtname = sctbl['event'][ii].item()

print(f"Event Name : {evtname}")
print(f"Model Name : {mdlname}")
print(f"Version    : {ver}")
print(f"Type       : {typ}")

Event Name : 1999em
Model Name : v19-1999em-corr
Version    : 1.0
Type       : IIP


In [28]:
source = sncosmo.get_source(mdlname, version=ver)
model = sncosmo.Model(source=source)
model.set(z=z)
model.set_source_peakabsmag(peakabsmag, peakabsfilter, 'ab')

In [29]:
tmin = model.mintime()
tmax = model.maxtime()
lmin = model.minwave()
lmax = model.maxwave()

print(f't          : {tmin:1.3f}-{tmax:1.3f} days')
print(f'wavelength : {lmin:1.3f}-{lmax:1.3f} A')

#	Wavelength [AA]
wavestep = 10
wave = np.arange(lmin, lmax, wavestep)

#	Time
timestep = 5
times = np.arange(-5, 50+timestep, timestep)

newfile = f'{path_model}/{mdlname}_v{ver}_type_{typ}_sn_synphot_{tag}.ecsv'

phtbl = make_sncosmo_model2synphot(
	model=model,
	speclite_class=lsst,
	filterlist=filterlist,
	bandlam=lsstlam,
	bandwidth=lsstbdw,
	times=times,
	wave=wave,
	# newfile=newfile,
	)

phtbl.meta['classification'] = f'Type {typ} SN'
phtbl.meta['model name'] = mdlname
phtbl.meta['event name'] = evtname
phtbl.meta['type'] = typ
phtbl.write(newfile, format='ascii.ecsv', overwrite=True)

phtbl

t          : -6.601-191.639 days
wavelength : 1605.160-11001.100 A


t,u,g,r,i,z,y
d,mag(AB),mag(AB),mag(AB),mag(AB),mag(AB),mag(AB)
float64,float64,float64,float64,float64,float64,float64
-5.0,4.052,4.533,4.682,5.061,5.183,5.167
0.0,4.121,4.254,4.413,4.742,4.829,4.847
5.0,4.484,4.282,4.382,4.677,4.742,4.766
10.0,4.842,4.367,4.329,4.582,4.686,4.767
15.0,5.362,4.540,4.347,4.566,4.700,4.807
20.0,5.839,4.648,4.368,4.529,4.652,4.794
25.0,6.118,4.768,4.396,4.506,4.637,4.734
30.0,6.329,4.855,4.432,4.487,4.584,4.626


# Iteration

## LSST (ugrizy)

In [30]:
tag = 'lsst'
filterlist = ['u', 'g', 'r', 'i', 'z', 'y']
peakabsfilter = 'bessellb'
peakabsmag = -19.0
zlist = [1e-4, 1e-3, 1e-2, 1e-1, 1e-0]
# zz = 0
# z = zlist[zz]
# ii = 0

for zz, z in enumerate(zlist):
	for ii in range(len(sctbl)):
		mdlname = sctbl['model'][ii].item()
		ver = str(sctbl['version'][ii].item())
		typ = sctbl['type'][ii].item()
		evtname = sctbl['event'][ii].item()

		print(f"[{ii+1}/{len(sctbl)}]\t@z={z:1.3f}")
		print(f"Event Name : {evtname}")
		print(f"Model Name : {mdlname}")
		print(f"Version    : {ver}")
		print(f"Type       : {typ}")
		#
		source = sncosmo.get_source(mdlname, version=ver)
		model = sncosmo.Model(source=source)
		model.set(z=z)
		model.set_source_peakabsmag(peakabsmag, peakabsfilter, 'ab')
		#
		tmin = model.mintime()
		tmax = model.maxtime()
		lmin = model.minwave()
		lmax = model.maxwave()

		print(f't          : {tmin:1.3f}-{tmax:1.3f} days')
		print(f'wavelength : {lmin:1.3f}-{lmax:1.3f} A')

		#	Wavelength [AA]
		wavestep = 10
		wave = np.arange(lmin, lmax, wavestep)

		#	Time
		timestep = 5
		times = np.arange(-5, 50+timestep, timestep)

		newfile = f'{path_model}/{mdlname}_v{ver}_type_{typ}_sn_z{z}_synphot_{tag}.ecsv'

		phtbl = make_sncosmo_model2synphot(
			model=model,
			speclite_class=lsst,
			filterlist=filterlist,
			bandlam=lsstlam,
			bandwidth=lsstbdw,
			times=times,
			wave=wave,
			# newfile=newfile,
			)

		phtbl.meta['classification'] = f'Type {typ} SN'
		phtbl.meta['model name'] = mdlname
		phtbl.meta['event name'] = evtname
		phtbl.meta['type'] = typ
		phtbl.meta['z'] = z
		phtbl.write(newfile, format='ascii.ecsv', overwrite=True)


[1/10]	@z=0.000
Event Name : 1999em
Model Name : v19-1999em-corr
Version    : 1.0
Type       : IIP
t          : -6.601-191.639 days
wavelength : 1605.160-11001.100 A
[2/10]	@z=0.000
Event Name : 2004et
Model Name : v19-2004et-corr
Version    : 1.0
Type       : IIP
t          : -14.501-243.524 days
wavelength : 1605.160-11001.100 A
[3/10]	@z=0.000
Event Name : Model
Model Name : nugent-sn2l
Version    : 1.2
Type       : IIL
t          : 0.000-411.041 days
wavelength : 1000.100-25002.500 A
[4/10]	@z=0.000
Event Name : 2007pk
Model Name : v19-2007pk-corr
Version    : 1.0
Type       : IIn
t          : -10.811-167.617 days
wavelength : 1605.160-11001.100 A
[5/10]	@z=0.000
Event Name : 2009ip
Model Name : v19-2009ip-corr
Version    : 1.0
Type       : IIn
t          : -12.001-86.809 days
wavelength : 1605.160-11001.100 A
[6/10]	@z=0.000
Event Name : 2004gt
Model Name : v19-2004gt-corr
Version    : 1.0
Type       : Ic
t          : -19.852-178.308 days
wavelength : 1605.160-11001.100 A
[7/10]	@

## Med-bands

In [31]:
tag = 'med'
filterlist = [filte.split('-')[-1] for filte in meds.names]
peakabsfilter = 'bessellb'
peakabsmag = -19.0
zlist = [1e-4, 1e-3, 1e-2, 1e-1, 1e-0]
# zz = 0
# z = zlist[zz]
# ii = 0

for zz, z in enumerate(zlist):
	for ii in range(len(sctbl)):
		mdlname = sctbl['model'][ii].item()
		ver = str(sctbl['version'][ii].item())
		typ = sctbl['type'][ii].item()
		evtname = sctbl['event'][ii].item()

		print(f"[{ii+1}/{len(sctbl)}]\t@z={z:1.3f}")
		print(f"Event Name : {evtname}")
		print(f"Model Name : {mdlname}")
		print(f"Version    : {ver}")
		print(f"Type       : {typ}")
		#
		source = sncosmo.get_source(mdlname, version=ver)
		model = sncosmo.Model(source=source)
		model.set(z=z)
		model.set_source_peakabsmag(peakabsmag, peakabsfilter, 'ab')
		#
		tmin = model.mintime()
		tmax = model.maxtime()
		lmin = model.minwave()
		lmax = model.maxwave()

		print(f't          : {tmin:1.3f}-{tmax:1.3f} days')
		print(f'wavelength : {lmin:1.3f}-{lmax:1.3f} A')

		#	Wavelength [AA]
		wavestep = 10
		wave = np.arange(lmin, lmax, wavestep)

		#	Time
		timestep = 5
		times = np.arange(-5, 50+timestep, timestep)

		newfile = f'{path_model}/{mdlname}_v{ver}_type_{typ}_sn_z{z}_synphot_{tag}.ecsv'

		phtbl = make_sncosmo_model2synphot(
			model=model,
			speclite_class=meds,
			filterlist=filterlist,
			bandlam=mlam,
			bandwidth=mbdw,
			times=times,
			wave=wave,
			# newfile=newfile,
			)

		phtbl.meta['classification'] = f'Type {typ} SN'
		phtbl.meta['model name'] = mdlname
		phtbl.meta['event name'] = evtname
		phtbl.meta['type'] = typ
		phtbl.meta['z'] = z
		phtbl.write(newfile, format='ascii.ecsv', overwrite=True)


[1/10]	@z=0.000
Event Name : 1999em
Model Name : v19-1999em-corr
Version    : 1.0
Type       : IIP
t          : -6.601-191.639 days
wavelength : 1605.160-11001.100 A
[2/10]	@z=0.000
Event Name : 2004et
Model Name : v19-2004et-corr
Version    : 1.0
Type       : IIP
t          : -14.501-243.524 days
wavelength : 1605.160-11001.100 A
[3/10]	@z=0.000
Event Name : Model
Model Name : nugent-sn2l
Version    : 1.2
Type       : IIL
t          : 0.000-411.041 days
wavelength : 1000.100-25002.500 A
[4/10]	@z=0.000
Event Name : 2007pk
Model Name : v19-2007pk-corr
Version    : 1.0
Type       : IIn
t          : -10.811-167.617 days
wavelength : 1605.160-11001.100 A
[5/10]	@z=0.000
Event Name : 2009ip
Model Name : v19-2009ip-corr
Version    : 1.0
Type       : IIn
t          : -12.001-86.809 days
wavelength : 1605.160-11001.100 A
[6/10]	@z=0.000
Event Name : 2004gt
Model Name : v19-2004gt-corr
Version    : 1.0
Type       : Ic
t          : -19.852-178.308 days
wavelength : 1605.160-11001.100 A
[7/10]	@

## Johnson Cousin (BVRI)

In [32]:
tag = 'jc'
# filterlist = [filte.split('-')[-1] for filte in jc.names]
filterlist = ['B', 'V', 'R', 'I']
peakabsfilter = 'bessellb'
peakabsmag = -19.0
zlist = [1e-4, 1e-3, 1e-2, 1e-1, 1e-0]
# zz = 0
# z = zlist[zz]
# ii = 0

for zz, z in enumerate(zlist):
	for ii in range(len(sctbl)):
		mdlname = sctbl['model'][ii].item()
		ver = str(sctbl['version'][ii].item())
		typ = sctbl['type'][ii].item()
		evtname = sctbl['event'][ii].item()

		print(f"[{ii+1}/{len(sctbl)}]\t@z={z:1.3f}")
		print(f"Event Name : {evtname}")
		print(f"Model Name : {mdlname}")
		print(f"Version    : {ver}")
		print(f"Type       : {typ}")
		#
		source = sncosmo.get_source(mdlname, version=ver)
		model = sncosmo.Model(source=source)
		model.set(z=z)
		model.set_source_peakabsmag(peakabsmag, peakabsfilter, 'ab')
		#
		tmin = model.mintime()
		tmax = model.maxtime()
		lmin = model.minwave()
		lmax = model.maxwave()

		print(f't          : {tmin:1.3f}-{tmax:1.3f} days')
		print(f'wavelength : {lmin:1.3f}-{lmax:1.3f} A')

		#	Wavelength [AA]
		wavestep = 10
		wave = np.arange(lmin, lmax, wavestep)

		#	Time
		timestep = 5
		times = np.arange(-5, 50+timestep, timestep)

		newfile = f'{path_model}/{mdlname}_v{ver}_type_{typ}_sn_z{z}_synphot_{tag}.ecsv'

		phtbl = make_sncosmo_model2synphot(
			model=model,
			speclite_class=jc,
			filterlist=filterlist,
			bandlam=jclam,
			bandwidth=jcbdw,
			times=times,
			wave=wave,
			# newfile=newfile,
			)

		phtbl.meta['classification'] = f'Type {typ} SN'
		phtbl.meta['model name'] = mdlname
		phtbl.meta['event name'] = evtname
		phtbl.meta['type'] = typ
		phtbl.meta['z'] = z
		phtbl.write(newfile, format='ascii.ecsv', overwrite=True)


[1/10]	@z=0.000
Event Name : 1999em
Model Name : v19-1999em-corr
Version    : 1.0
Type       : IIP
t          : -6.601-191.639 days
wavelength : 1605.160-11001.100 A
[2/10]	@z=0.000
Event Name : 2004et
Model Name : v19-2004et-corr
Version    : 1.0
Type       : IIP
t          : -14.501-243.524 days
wavelength : 1605.160-11001.100 A
[3/10]	@z=0.000
Event Name : Model
Model Name : nugent-sn2l
Version    : 1.2
Type       : IIL
t          : 0.000-411.041 days
wavelength : 1000.100-25002.500 A
[4/10]	@z=0.000
Event Name : 2007pk
Model Name : v19-2007pk-corr
Version    : 1.0
Type       : IIn
t          : -10.811-167.617 days
wavelength : 1605.160-11001.100 A
[5/10]	@z=0.000
Event Name : 2009ip
Model Name : v19-2009ip-corr
Version    : 1.0
Type       : IIn
t          : -12.001-86.809 days
wavelength : 1605.160-11001.100 A
[6/10]	@z=0.000
Event Name : 2004gt
Model Name : v19-2004gt-corr
Version    : 1.0
Type       : Ic
t          : -19.852-178.308 days
wavelength : 1605.160-11001.100 A
[7/10]	@